<a href="https://colab.research.google.com/github/ryaltic/LCS-Predictions-2026/blob/main/LCS_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Getting data to use for the model

import pandas as pd
import requests

url = "https://gol.gg/game/stats/73776/page-fullstats/"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)

tables = pd.read_html(response.text)

df = tables[0]
print(df.head())





  Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0     Player     Morgan  Josedeodo       Quid       Yeon     CoreJJ   
1       Role        TOP     JUNGLE        MID        ADC    SUPPORT   
2      Level         17         15         18         14         13   
3      Kills          3          4          2          2          0   
4     Deaths          1          1          1          1          1   

  Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  
0      Fudge  Contractz      Zinie       Bvoy        Ceos  
1        TOP     JUNGLE        MID        ADC     SUPPORT  
2         16         14         17         13          11  
3          1          1          2          1           0  
4          2          2          3          2           2  


/tmp/ipython-input-3597712566.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


In [14]:
# 1) Set first column as index (stat names)
df = df.set_index(df.columns[0])

# 2) Transpose so players become rows
df = df.T

# 3) Reset index
df = df.reset_index(drop=True)

print(df.head())




Unnamed: 0     Player     Role Level Kills Deaths Assists KDA   CS  \
0              Morgan      TOP    17     3      1       2   5  257   
1           Josedeodo   JUNGLE    15     4      1       7  11  210   
2                Quid      MID    18     2      1       5   7  298   
3                Yeon      ADC    14     2      1       7   9  274   
4              CoreJJ  SUPPORT    13     0      1      10  10   23   

Unnamed: 0 CS in Team's Jungle CS in Enemy Jungle  ... Damage self mitigated  \
0                            4                NaN  ...                 28434   
1                          185                NaN  ...                 29846   
2                           12                NaN  ...                 12960   
3                            4                NaN  ...                 14691   
4                            0                NaN  ...                  2995   

Unnamed: 0 Total Damage Shielded On Teammates Time ccing others  \
0                              

In [15]:
df.columns

Index(['Player', 'Role', 'Level', 'Kills', 'Deaths', 'Assists', 'KDA', 'CS',
       'CS in Team's Jungle', 'CS in Enemy Jungle', 'CSM', 'Golds', 'GPM',
       'GOLD%', 'Vision Score', 'Wards placed', 'Wards destroyed',
       'Control Wards Purchased', 'Detector Wards Placed', 'VSPM', 'WPM',
       'VWPM', 'WCPM', 'VS%', 'Total damage to Champion', 'Physical Damage',
       'Magic Damage', 'True Damage', 'DPM', 'DMG%', 'K+A Per Minute', 'KP%',
       'Solo kills', 'Double kills', 'Triple kills', 'Quadra kills',
       'Penta kills', 'GD@15', 'CSD@15', 'XPD@15', 'LVLD@15',
       'Objectives Stolen', 'Damage dealt to turrets',
       'Damage dealt to buildings', 'Total heal', 'Total Heals On Teammates',
       'Damage self mitigated', 'Total Damage Shielded On Teammates',
       'Time ccing others', 'Total Time CC Dealt', 'Total damage taken',
       'Total Time Spent Dead', 'Consumables purchased', 'Items Purchased',
       'Shutdown bounty collected', 'Shutdown bounty lost'],
      dt

In [16]:
df.columns.name = None


In [17]:
df.head()

,Player,Role,Level,Kills,Deaths,Assists,KDA,CS,CS in Team's Jungle,CS in Enemy Jungle,...,Damage self mitigated,Total Damage Shielded On Teammates,Time ccing others,Total Time CC Dealt,Total damage taken,Total Time Spent Dead,Consumables purchased,Items Purchased,Shutdown bounty collected,Shutdown bounty lost
0,Morgan,TOP,17,3,1,2,5,257,4,NaN,...,28434,0,7,161,16980,37,6,21,0,0
1,Josedeodo,JUNGLE,15,4,1,7,11,210,185,NaN,...,29846,0,13,542,23862,46,10,30,89,589
2,Quid,MID,18,2,1,5,7,298,12,NaN,...,12960,0,13,47,18117,25,7,25,0,0
3,Yeon,ADC,14,2,1,7,9,274,4,NaN,...,14691,0,4,45,22055,36,5,29,0,38
4,CoreJJ,SUPPORT,13,0,1,10,10,23,0,NaN,...,2995,2086,16,229,6989,12,14,32,0,0


In [ ]:
import pandas as pd
import requests

headers = {"User-Agent": "Mozilla/5.0"}

def pull_match(url, match_id, game_num, team_map, winner):

    response = requests.get(url, headers=headers)
    tables = pd.read_html(response.text)

    df = tables[0]

    # Clean table
    df = df.set_index(df.columns[0]).T.reset_index(drop=True)
    df.columns.name = None
    df.index.name = None

    # Add metadata
    df["MatchID"] = match_id
    df["Game"] = game_num

    # Assign teams (first 5 vs second 5)
    df["Team"] = df.index.map(lambda x: team_map[0] if x < 5 else team_map[1])

    # Winner label
    df["Win"] = df["Team"].apply(lambda x: 1 if x == winner else 0)

    return df


In [ ]:
all_matches = []

urls = [
    "https://gol.gg/game/stats/73776/page-fullstats/",
    "https://gol.gg/game/stats/73777/page-fullstats/"
]

for i, url in enumerate(urls):

    df = pull_match(
        url=url,
        match_id=i,
        game_num=1,
        team_map=("TL","C9"),
        winner="TL"
    )

    all_matches.append(df)

final_df = pd.concat(all_matches, ignore_index=True)
